In [ ]:
import os
import pandas as pd
from pepars.utils import Illumina_FASTQ_File_Set
from pepars.utils import DNA as DNA_utils
from pepars.utils import FASTQ as FASTQ_utils
from pepars.analysis import DNA as DNA_analysis

from Bio.Seq import reverse_complement 

Function to count the prevalence of sequences withing bulk sequencing data. 

In [ ]:
def count_seq(sample_names, lower_nt, higher_nt, read):
    
    index = 0

    for sample in sample_names:

        file_set = Illumina_FASTQ_File_Set(file_path, sample)

        seq_counts = {}

        for sequences in file_set.get_sequence_iterator():

            sequence = sequences[read][lower_nt:higher_nt]

            if sequence not in seq_counts:
                seq_counts[sequence] = 1
            else:
                seq_counts[sequence] += 1
                        
        sorted_seq_counts = [(sequence, count) for sequence, count in sorted(seq_counts.items(), key=lambda x: x[1], reverse=True)]
        file_set.close()

        collapsed_seq_counts = DNA_analysis.collapse_similar_sequences([(x, y) for x, y in seq_counts.items()])
        sorted_collapsed_seq_counts = [(sequence, count) for sequence, count in sorted(collapsed_seq_counts, key=lambda x: x[1], reverse=True)]

        path = "analysis/"

        outfilename = path + sample + "_counts.csv"

        df = pd.DataFrame(sorted_collapsed_seq_counts, columns = ['sequence', 'counts'])
        
        aa_seq = np.empty_like(df['sequence'])
        
        aa_rep_counts = {}
        
        for i, barcode in enumerate(df['sequence']):
            aa_seq[i] = Bio.Seq.translate(df['sequence'][i])
        
        df['aa_sequence'] = aa_seq.astype('str')
        
        df.to_csv(outfilename, index=False)

        print(index)
        index += 1

Set the file path to FASTQ files, the nucleotides to examine, and the corresponding sample names. Example given is for substitution between AA 492-498.

In [ ]:
file_path = 'raw'

lower_nt = 18 # 29 for substitution between 452-458 or 585-591
higher_nt = 39 # 50 for substitution between 452-458 or 585-591

sample_names = ['492ll', 
                '492vg']

count_seq(sample_names, lower_nt, higher_nt, 0)